# PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
* Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. 

* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

* Q12: Find the facilities with their usage by member, but not guests

* Q13: Find the facilities usage by month, but not guests

In [14]:
import sqlite3
import pandas as pd

# connect to local database
connect = sqlite3.connect('/Users/katialopes-gilbert/repos/springboard-projects/sql-project/sqlite_db_pythonsqlite.db')

# create cursor object to interact with database
cursor = connect.cursor()

In [15]:
# Define the query
bookings_view = '''
SELECT *
FROM Bookings
'''

# Execute the query and fetch the results into a DataFrame
bookings = pd.read_sql_query(bookings_view, connect)

# Display the results
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [16]:
# Define the query
facilities_view = '''
SELECT *
FROM Facilities
'''

# Execute the query and fetch the results into a DataFrame
facilities = pd.read_sql_query(facilities_view, connect)

# Display the results
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [17]:
# Define the query
members_view = '''
SELECT *
FROM Members
'''

# Execute the query and fetch the results into a DataFrame
members = pd.read_sql_query(members_view, connect)

# Display the results
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


## Question 10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue.

In [22]:
facility_rev = '''
SELECT s.facility_name, SUM(s.booking_cost) AS total_rev
FROM (
    SELECT f.name AS facility_name, 
    	CASE WHEN b.memid = 0 THEN (f.guestcost * b.slots)
    		 WHEN b.memid != 0 THEN (f.membercost * b.slots)
    END AS booking_cost
    FROM `Bookings` AS b
    INNER JOIN `Facilities` AS f
    ON b.facid = f.facid) AS s
GROUP BY s.facility_name
HAVING total_rev < 1000
ORDER BY total_rev DESC
'''

rev_df = pd.read_sql_query(facility_rev, connect)

rev_df

,facility_name,total_rev
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


## Question 11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [38]:
member_recs = '''
SELECT m1.surname || ', ' || m1.firstname AS member_name, 
    (SELECT m2.surname || ', ' || m2.firstname
    FROM Members AS m2
    WHERE m2.memid = m1.recommendedby) AS recommender
FROM Members AS m1
WHERE m1.recommendedby >= 1
ORDER BY member_name
'''

recommendations_df = pd.read_sql_query(member_recs, connect)

recommendations_df

,member_name,recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


## Question 12: Find the facilities with their usage by member, but not guests

In [48]:
facility_usage_members = '''
SELECT COUNT(b.bookid) AS total_bookings, m.surname || ', ' || m.firstname AS member_name, f.name AS facility_name
FROM Bookings AS b
INNER JOIN Members AS m
ON b.memid = m.memid
INNER JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY facility_name, member_name
ORDER BY member_name, total_bookings DESC
'''

facility_usage_members_df = pd.read_sql_query(facility_usage_members, connect)

facility_usage_members_df

,total_bookings,member_name,facility_name
0,42,"Bader, Florence",Table Tennis
1,33,"Bader, Florence",Snooker Table
2,23,"Bader, Florence",Pool Table
3,9,"Bader, Florence",Badminton Court
4,8,"Bader, Florence",Tennis Court 2
...,...,...,...
197,1,"Tupperware, Hyacinth",Squash Court
198,33,"Worthington-Smyth, Henry",Pool Table
199,4,"Worthington-Smyth, Henry",Badminton Court
200,3,"Worthington-Smyth, Henry",Table Tennis


## Question 13: Find the facilities usage by month, but not guests

In [53]:
facility_usage_month = '''
SELECT f.name AS facility_name, strftime('%m',b.starttime) AS "month", COUNT(b.bookid) AS total_bookings
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY facility_name, month
ORDER BY facility_name, total_bookings DESC
'''

facility_usage_month_df = pd.read_sql_query(facility_usage_month, connect)

facility_usage_month_df

,facility_name,month,total_bookings
0,Badminton Court,09,161
1,Badminton Court,08,132
2,Badminton Court,07,51
3,Massage Room 1,09,191
4,Massage Room 1,08,153
5,Massage Room 1,07,77
6,Massage Room 2,09,14
7,Massage Room 2,08,9
8,Massage Room 2,07,4
9,Pool Table,09,408
